<a href="https://colab.research.google.com/github/kbw612/Fantasy/blob/main/colab/AnalyzeTeamPreviousGames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy.lib.function_base import median
from pandas.core.frame import DataFrame
import pandas as pd

dark_green_color = "\033[32m"
green_color = "\033[92m"
red_color = "\033[91m"
default_color = "\033[0m"
bold_text = "\033[1m"
default_text = "\033[0m"

styles = [
    {
        'selector': 'th',
        'props': [
            ('text-align', 'left')  # Align headers to the left
        ]
    }
]

fantasy_points_per_game_column_name = 'FPts/G'
position_column_name = 'Pos'
team_column_name = 'Team'
average_diff_column_name = 'Avg Diff'
median_diff_column_name = 'Median Diff'
average_fantasy_points_per_game_column_name = 'Avg FPts/G'
median_fantasy_points_per_game_column_name = 'Median FPts/G'

pd.set_option('display.max_colwidth', 40)

team_nickname_to_team_abbreviation_map = {
  "Cardinals": "ARI",
  "Falcons": "ATL",
  "Ravens": "BAL",
  "Bills": "BUF",
  "Panthers": "CAR",
  "Bears": "CHI",
  "Bengals": "CIN",
  "Browns": "CLE",
  "Cowboys": "DAL",
  "Broncos": "DEN",
  "Lions": "DET",
  "Packers": "GB",
  "Texans": "HOU",
  "Colts": "IND",
  "Jaguars": "JAX",
  "Chiefs": "KC",
  "Chargers": "LAC",
  "Rams": "LAR",
  "Raiders": "LV",
  "Dolphins": "MIA",
  "Vikings": "MIN",
  "Patriots": "NE",
  "Saints": "NO",
  "Giants": "NYG",
  "Jets": "NYJ",
  "Eagles": "PHI",
  "Steelers": "PIT",
  "49ers": "SF",
  "Seahawks": "SEA",
  "Buccaneers": "TB",
  "Titans": "TEN",
  "Commanders": "WAS"
}

team_full_name_to_team_abbreviation_map = {
  "Arizona": "ARI",
  "Atlanta": "ATL",
  "Baltimore": "BAL",
  "Buffalo": "BUF",
  "Carolina": "CAR",
  "Chicago": "CHI",
  "Cincinnati": "CIN",
  "Cleveland": "CLE",
  "Dallas": "DAL",
  "Denver": "DEN",
  "Detroit": "DET",
  "Green Bay": "GB",
  "Houston": "HOU",
  "Indianapolis": "IND",
  "Jacksonville": "JAX",
  "Kansas City": "KC",
  "LA Chargers": "LAC",
  "LA Rams": "LAR",
  "Las Vegas": "LV",
  "Miami": "MIA",
  "Minnesota": "MIN",
  "New England": "NE",
  "New Orleans": "NO",
  "NY Giants": "NYG",
  "NY Jets": "NYJ",
  "Philadelphia": "PHI",
  "Pittsburgh": "PIT",
  "San Francisco": "SF",
  "Seattle": "SEA",
  "Tampa Bay": "TB",
  "Tennessee": "TEN",
  "Washington": "WAS"
}

def get_team_abbreviation(nickname):
  team_abbrev = team_nickname_to_team_abbreviation_map.get(nickname, "")

  if team_abbrev == "":
        return nickname

  return team_abbrev

def get_team_abbreviation_by_full_name(full_name):
  team_abbrev = team_full_name_to_team_abbreviation_map.get(full_name, "")

  if team_abbrev == "":
        return full_name

  return team_abbrev


def get_games(week, start_date, end_date):
  games_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DKSalaries-week{week}.csv")

  games_df = games_df.drop(columns=['Position', 'Name + ID', 'Name', 'ID', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame'])

  split_columns = ['Game', 'Date', 'Time', 'TimeZone']
  games_df[split_columns] = games_df['Game Info'].str.split(" ", expand=True)
  games_df = games_df.drop_duplicates(subset=['Game Info', 'Game', 'Date', 'Time', 'TimeZone'])
  games_df = games_df.sort_values(by=['Date', 'Time', 'Game'], ascending=[True, True, True])
  games_df[['Away Team', 'Home Team']] = games_df['Game'].str.split("@", expand=True)

  games_df['Datetime'] = pd.to_datetime(games_df['Date'] + ' ' + games_df['Time'])

  games_df = games_df[(games_df['Datetime'] >= start_date) & (games_df['Datetime'] <= end_date)]
  games_df = games_df.sort_values(by=['Date', 'Time', 'Game'], ascending=[True, True, True])

  split_columns.append('Datetime')
  games_df = games_df.drop(columns=split_columns)

  #print(games_df)

  return games_df

def get_all_teams(filename, week):
  df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/stats/{filename}.csv")

  df = df[df["Week"] == week]

  for row in df.index:
    df.at[row, "Team"] = get_team_abbreviation(df.at[row, "Team"])
#Week       Team Pos  G   Cmp  Passing Attempts Passing Yards  Passing TDs  INTs  Rushing Attempts  Rushing Yards  Rushing TDs  Receptions Receiving Yards  Receiving TDs  FPts  FPts/G
  return df

def get_point_stats_by_position(df):
  median_position_points = []
  average_position_points = []
  positions = ['QB', 'RB', 'WR', 'TE']

  for position in positions:
    position_points_df = df[df[position_column_name] == position]
    median_position_points.append(round(position_points_df[fantasy_points_per_game_column_name].median(axis=0), 2))
    average_position_points.append(round(position_points_df[fantasy_points_per_game_column_name].mean(axis=0), 2))

  return pd.DataFrame({position_column_name: positions, average_diff_column_name: [0, 0, 0, 0], median_diff_column_name: [0, 0, 0, 0], average_fantasy_points_per_game_column_name: average_position_points, median_fantasy_points_per_game_column_name: median_position_points})

#def format_points_diff_column_value(points):
#  if points < 0:
#    return f'{red_color}{points}{default_color}'
#  elif points > 6:
#    return f'{dark_green_color}{points}{default_color}'
#  elif points > 0:
#    return f'{green_color}{points}{default_color}'
 # return points

def print_team_points(header, team_points_df, point_stats_df):
  print("")
  print(f"{bold_text}{header}{default_text}")
  team_points_df = pd.merge(team_points_df[display_columns], point_stats_df, left_on='Pos', right_on='Pos', how='left')

  for row in team_points_df.index:
    team_points_df.at[row, average_diff_column_name] = round(team_points_df.at[row, fantasy_points_per_game_column_name] - team_points_df.at[row, average_fantasy_points_per_game_column_name], 2)
    team_points_df.at[row, median_diff_column_name] = round(team_points_df.at[row, fantasy_points_per_game_column_name] - team_points_df.at[row, median_fantasy_points_per_game_column_name], 2)

  print(team_points_df.to_string(index=False))

def get_team_stats(filename, week):
  df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/stats/{filename}.csv?1=3")
  df = df[df["Week"] == week]

  for row in df.index:
    df.at[row, "Team"] = get_team_abbreviation_by_full_name(df.at[row, "Team"])

  return df

def print_team_stats(header, team_abbrev, team_stats_df):
  print("")
  print(f"{bold_text}{header}{default_text}")

  team_stats_df = team_stats_df[team_stats_df["Team"] == team_abbrev]
  #team_points_df = pd.merge(team_points_df[display_columns], point_stats_df, left_on='Pos', right_on='Pos', how='left')

  #for row in team_points_df.index:
  #  team_points_df.at[row, average_diff_column_name] = round(team_points_df.at[row, fantasy_points_per_game_column_name] - team_points_df.at[row, average_fantasy_points_per_game_column_name], 2)
  #  team_points_df.at[row, median_diff_column_name] = round(team_points_df.at[row, fantasy_points_per_game_column_name] - team_points_df.at[row, median_fantasy_points_per_game_column_name], 2)

  print(team_stats_df.to_string(index=False))

def format_text(value):
    return f'{{: <{100}}}'.format(value)

def print_team_game_recaps(header, team_abbrev, game_recaps_df):
  print("")
  print(f"{bold_text}{header}{default_text}")

  game_recaps_df = game_recaps_df[(game_recaps_df["Away"] == team_abbrev) | (game_recaps_df["Home"] == team_abbrev)]

  game_recaps_df.style.format({'Game Summary': format_text})

  #game_recaps_df['Game Summary'] = game_recaps_df['Game Summary'].apply(lambda x: x.ljust(2))

  print(game_recaps_df.to_string(index=False))

# last 5 games, print the players at each postion???
# last 5 games, print game recap????

current_week = 18
previous_week = current_week - 1
nbr_to_print = 1000
start_contest_date="01/06/2024 01:00PM"
end_contest_date="01/07/2024 04:30PM"

display_columns = [position_column_name, fantasy_points_per_game_column_name]

pd.set_option('display.max_colwidth', None)  # Disable column width limit
pd.set_option('display.expand_frame_repr', False)

all_teams_points_allowed_df = get_all_teams("team-points-allowed", current_week)
all_teams_points_scored_df = get_all_teams("team-points-scored", current_week)

point_stats_allowed_df = get_point_stats_by_position(all_teams_points_allowed_df)
point_stats_scored_df = get_point_stats_by_position(all_teams_points_scored_df)

defense_rush_yards_per_attempt_allowed_df = get_team_stats("Team-Defense-Opponent-Yards-per-Rush-Attempt-Allowed", current_week)
offensive_rush_yards_per_attempt_df =  get_team_stats("Team-Offensive-Yards-per-Rush-Attempt", current_week)
defense_pass_yards_per_completion_allowed_df = get_team_stats("Team-Defense-Opponent-Yards-per-Pass-Completion-Allowed", current_week)
offensive_pass_yards_per_completion_df =  get_team_stats("Team-Offensive-Yards-per-Pass-Completion", current_week)

game_recaps_df = pd.read_csv("https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/Game-Recaps.csv")

#dk_players_df = pd.read_csv("https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DK-Players.csv")
games_df = get_games(week=current_week, start_date = start_contest_date, end_date = end_contest_date)

for index, game_row in games_df.iterrows():
  print('')
  print(f"{bold_text}{game_row['Game Info']}{default_text}")
  away_team = game_row['Away Team']
  home_team = game_row['Home Team']

  print_team_game_recaps(f'{away_team} Game Recaps', away_team, game_recaps_df)
  print_team_game_recaps(f'{home_team} Game Recaps', home_team, game_recaps_df)

  print_team_stats(f'{away_team} Defense Opponent Yards per Rush Attempt Allowed', away_team, defense_rush_yards_per_attempt_allowed_df)
  print_team_stats(f'{home_team} Offensive Yards per Rush Attempt', home_team, offensive_rush_yards_per_attempt_df)

  print_team_stats(f'{away_team} Offensive Yards per Rush Attempt', away_team, offensive_rush_yards_per_attempt_df)
  print_team_stats(f'{home_team} Defense Opponent Yards per Rush Attempt Allowed', home_team, defense_rush_yards_per_attempt_allowed_df)

  print_team_stats(f'{away_team} Defense Opponent Yards per Pass Completion Allowed', away_team, defense_pass_yards_per_completion_allowed_df)
  print_team_stats(f'{home_team} Offensive Yards per Pass Completion', home_team, offensive_pass_yards_per_completion_df)

  print_team_stats(f'{away_team} Offensive Yards per Pass Completion', away_team, offensive_pass_yards_per_completion_df)
  print_team_stats(f'{home_team} Defense Opponent Yards per Pass Completion Allowed', home_team, defense_pass_yards_per_completion_allowed_df)

  away_team_points_allowed_df = all_teams_points_allowed_df[all_teams_points_allowed_df[team_column_name] == away_team]
  home_team_points_allowed_df = all_teams_points_allowed_df[all_teams_points_allowed_df[team_column_name] == home_team]
  away_team_points_scored_df = all_teams_points_scored_df[all_teams_points_scored_df[team_column_name] == away_team]
  home_team_points_scored_df = all_teams_points_scored_df[all_teams_points_scored_df[team_column_name] == home_team]

  print_team_points(f'{away_team} Points Allowed', away_team_points_allowed_df, point_stats_allowed_df)
  print_team_points(f'{home_team} Points Scored', home_team_points_scored_df, point_stats_allowed_df)

  print_team_points(f'{away_team} Points Scored', away_team_points_scored_df, point_stats_allowed_df)
  print_team_points(f'{home_team} Points Allowed', home_team_points_allowed_df, point_stats_allowed_df)




HOU@IND 01/06/2024 08:15PM ET

HOU Game Recaps
 Week Away Home     Score                                                                                        Game Summary
    4  PIT  HOU  HOU 30-6                                  Half: Hou 16-0  4th qtr: Hou two passing TDs.  Picket knee injury.
    5  HOU  ATL ATL 21-19                              1st qtr: ATL 7-3. Half: HOU 9-7. 3rd qtr: HOU 12-7 4th qtr: ATL 21-19.
    6   NO  HOU HOU 20-13                            1st qtr: 7-7  2nd qtr: HOU 17-10  3rd qtr: HOU 20-13  4th qtr: HOU 20-13
    8  HOU  CAR CAR 15-13                              1st qtr: 0-0  2nd qtr: HOU 7-6  3rd qtr: HOU 13-12  4th qtr: CAR 15-13
    9   TB  HOU HOU 39-37                          1st qtr: TB 10-7  2nd qtr: TB 17-10  3rd qtr: TB 23-22  4th qtr: HOU 39-37
   10  HOU  CIN HOU 30-27                         1st qtr: CIN 7-0  2nd qtr: HOU 10-7  3rd qtr: HOU 20-17  4th qtr: HOU 30-27
   11  ARI  HOU HOU 21-16                       1st qtr: ARI 10-7  2nd